# M4 | Research Investigation Notebook

In this notebook, you will do a research investigation of your chosen dataset in teams. You will begin by formally selecting your research question (task 0), then processing your data (task 1), creating a predictive model (task 2), and evaluating your model's results (task 3).

Please upload your solved notebook to Moodle (under [Milestone 4 Submission](https://moodle.epfl.ch/mod/assign/view.php?id=1199557)) adding your team name in title, example: `m4-lernnavi-teamname.ipynb`. Please run all cells before submission so we can grade effectively.


## Brief overview of Lernnavi
[Lernnavi](https://www.lernnavi.ch) is an instrument for promoting part of the basic technical study skills in German and mathematics.

Lernnavi's dataset is formatted in three main tables:
* *users*: demographic information of users.
* *events*: events done by the users in the platform.
* *transactions*: question and answer solved by user.

You should provide arguments and justifications for all of your design decisions throughout this investigation. You can use your M3 responses as the basis for this discussion.

In [1]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATA_DIR = './data' # You many change the directory

users = pd.read_csv('{}/users.csv.gz'.format(DATA_DIR))
events = pd.read_csv('{}/events.csv.gz'.format(DATA_DIR))
transactions = pd.read_csv('{}/transactions.csv.gz'.format(DATA_DIR))

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

## Task 0: Research Question

**Research question:**
*Predicting user motivation and retention. E.g., dropout or long pauses prediction, level checks prediction (self-supervised learning)*

## Task 1: Data Preprocessing

In this section, you are asked to preprocess your data in a way that is relevant for the model. Please include 1-2 visualizations of features / data explorations that are related to your downstream prediction task.

In [3]:
# Your code for data processing goes here
events['date'] = pd.to_datetime(events['timestamp'], unit='ms')
events["week"] = events["date"].dt.isocalendar().week
events["year"] = events["date"].dt.year

In [4]:
df_weekly = events[["user_id","year","week"]].dropna().groupby(["user_id","year","week"]).size().reset_index(name='num_events')
df_weekly

,user_id,year,week,num_events
0,387604,2021,20,1
1,387604,2021,21,1
2,387604,2021,25,7
3,387604,2021,26,25
4,387604,2021,31,12
...,...,...,...,...
27527,404600,2022,8,34
27528,404603,2022,8,112
27529,404604,2022,8,43
27530,404605,2022,8,22


In [5]:
all_questions = events[["user_id","year","week","action","transaction_token"]].dropna()
all_questions = all_questions[all_questions['action']=='SUBMIT_ANSWER']

evaluations = all_questions.merge(transactions[['transaction_token', 'evaluation']].dropna().replace(["CORRECT","PARTIAL","WRONG"],[1,0.5,0]),
                                  on='transaction_token',
                                  how='left')

# Remove users that have a transaction in the event table that is not in the transaction table (~27000 / 400000)
evaluations = evaluations[evaluations['evaluation'].notna()]
num_questions = evaluations.drop(columns = ["transaction_token"]).groupby(
    ["user_id","year","week"]).size().reset_index(name='num_questions')
evaluations_count = evaluations.drop(columns = ["transaction_token"]).groupby(
    ["user_id","year","week"]).sum().reset_index()

df_questions = num_questions.merge(evaluations_count, on=['user_id', 'year', 'week'], how='left')
df_questions['percentage_correct'] = df_questions['evaluation'] / df_questions['num_questions']
df_questions = df_questions.drop(columns='evaluation')
df_questions

,user_id,year,week,num_questions,percentage_correct
0,387604,2021,35,1,0.000000
1,387604,2021,38,4,0.125000
2,387604,2021,39,2,0.750000
3,387604,2021,40,1,0.500000
4,387604,2021,41,1,0.500000
...,...,...,...,...,...
18526,404597,2022,8,1,0.500000
18527,404598,2022,8,1,1.000000
18528,404599,2022,8,1,1.000000
18529,404603,2022,8,13,0.653846


In [6]:
num_theory = events[["user_id","year","week","action"]].dropna()
num_theory = num_theory[num_theory["action"]=='GO_TO_THEORY'].groupby(
    ["user_id","year","week"]).size().reset_index(name='num_theory')
num_theory

,user_id,year,week,num_theory
0,387604,2021,33,18
1,387604,2021,34,50
2,387604,2021,35,118
3,387604,2021,36,27
4,387604,2021,37,78
...,...,...,...,...
7858,404589,2022,8,4
7859,404597,2022,8,8
7860,404598,2022,8,15
7861,404599,2022,8,38


In [7]:
# Filter out uneeded users and merge all tables
df_weekly_merged = df_weekly.copy()
df_weekly_merged = df_weekly_merged.merge(df_questions, on=['user_id', 'year', 'week'], how='left').fillna(value={'num_questions':0})
df_weekly_merged = df_weekly_merged.merge(num_theory, on=['user_id', 'year', 'week'], how='left').fillna(value={'num_theory':0})

# Renumber weeks
df_weekly_merged = df_weekly_merged.sort_values(by = ["user_id","year","week"]).groupby('user_id').head(13).reset_index(drop=True)
df_weekly_merged["week"] = df_weekly_merged.groupby('user_id').cumcount()
df_weekly_merged = df_weekly_merged.drop(columns = ["year"])
df_weekly_merged

,user_id,week,num_events,num_questions,percentage_correct,num_theory
0,387604,0,1,0.0,NaN,0.0
1,387604,1,1,0.0,NaN,0.0
2,387604,2,7,0.0,NaN,0.0
3,387604,3,25,0.0,NaN,0.0
4,387604,4,12,0.0,NaN,0.0
...,...,...,...,...,...,...
26957,404600,0,34,0.0,NaN,0.0
26958,404603,0,112,13.0,0.653846,0.0
26959,404604,0,43,4.0,0.375000,0.0
26960,404605,0,22,0.0,NaN,0.0


In [8]:
# NaN values should be ignored by the Standarizer 
columns_to_standarize = ['num_events', 'num_questions', 'percentage_correct', 'num_theory']
scaler = StandardScaler()

df_weekly_stand = df_weekly_merged.copy()
df_weekly_stand[columns_to_standarize] = scaler.fit_transform(df_weekly_stand[columns_to_standarize]) 
df_weekly_stand

,user_id,week,num_events,num_questions,percentage_correct,num_theory
0,387604,0,-0.765258,-0.575455,NaN,-0.242192
1,387604,1,-0.765258,-0.575455,NaN,-0.242192
2,387604,2,-0.728122,-0.575455,NaN,-0.242192
3,387604,3,-0.616712,-0.575455,NaN,-0.242192
4,387604,4,-0.697175,-0.575455,NaN,-0.242192
...,...,...,...,...,...,...
26957,404600,0,-0.561008,-0.575455,NaN,-0.242192
26958,404603,0,-0.078233,-0.048811,0.024995,-0.242192
26959,404604,0,-0.505303,-0.413411,-1.496555,-0.242192
26960,404605,0,-0.635281,-0.575455,NaN,-0.242192


In [9]:
# USE THIS PIPELINE TO APPLY TRANSFORMATIONS TO ONLY SOME COLUMNS

# preprocessor = ColumnTransformer(
#     transformers=[('scaler', StandardScaler(), columns_to_standarize)],
#     remainder='passthrough'
# )

# pipeline = Pipeline([
#     ('preprocessor', preprocessor)
# ]) # add your model in pipeline

# pipeline.fit(THE_DATA)


*Your discussion about your processing decisions goes here*

## Task 2: Model Building

Train a model for your research question. 

In [10]:
# Your code for training a model goes here

*Your discussion about your model training goes here*

## Task 3: Model Evaluation
In this task, you will use metrics to evaluate your model.

In [11]:
# Your code for model evaluation goes here

*Your discussion/interpretation about your model's behavior goes here*